# **1. 따릉이 API**

* https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import warnings

warnings.filterwarnings('ignore')

### 1-1. 데이터 요청하기

In [ ]:
targetsite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'

request = requests.post(targetsite, data = {'stationGrpSeq' : 'ALL'})

# print(request)
# print(request.text)


### 1-2. json 데이터 처리하기

* json.loads() : json 타입의 문자열 데이터를 파이썬에서 처리 할수 있도록 변환(딕셔너리로 변환)

In [32]:
bike_json = json.loads(request.text)

# print(bike_json)
# print(type(bike_json))

### 1-3. 딕셔너리 타입의 데이터를 데이터 프레임으로 변환하기

* json_normalize() : 딕셔너리 자료구조의 타입의 데이터를 판다스 데이터프레임으로 변환

In [ ]:
bike_df = pd.json_normalize(bike_json, 'realtimeList')

bike_df.head()

,stationId,stationName,stationImgFileName,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,stationSeCd,mode
0,ST-4,102. 망원역 1번출구 앞,,126.91062927,37.55564880,15,0,10,2,RAK_002,None
1,ST-5,103. 망원역 2번출구 앞,,126.91083527,37.55495071,14,0,15,1,RAK_002,None
2,ST-6,104. 합정역 1번출구 앞,,126.91508484,37.55073929,13,0,0,0,RAK_002,None
3,ST-7,105. 합정역 5번출구 앞,,126.91482544,37.55000687,5,0,1,0,RAK_002,None
4,ST-8,106. 합정역 7번출구 앞,,126.91282654,37.54864502,12,0,2,1,RAK_002,None


In [ ]:
bike_df.columns

Index(['stationId', 'stationName', 'stationImgFileName', 'stationLongitude',
       'stationLatitude', 'rackTotCnt', 'parkingBikeTotCnt',
       'parkingQRBikeCnt', 'parkingELECBikeCnt', 'stationSeCd', 'mode'],
      dtype='object')

* stationname : 대여소 이름
* stationid : 고유한 대여소 번호
* stationlongitude : 대여소 경도
* stationlatitude : 대여소 위도
* racktotcnt : 주차 가능한 전체 자전거 대수
* parkingbiketotcnt : 주차된 따릉이 총 대수
* parkingqrbikecnt : 주차된 따릉이 qr형 총 대수
* parkingelecbikecnt : 주차된 새싹 따릉이 총 대수

In [ ]:
bike_df_map = bike_df[['stationId', 'stationName', 'stationLongitude',
                       'stationLatitude', 'rackTotCnt', 'parkingBikeTotCnt',
                       'parkingQRBikeCnt', 'parkingELECBikeCnt']]

bike_df_map.head()

,stationId,stationName,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt
0,ST-4,102. 망원역 1번출구 앞,126.91062927,37.55564880,15,0,10,2
1,ST-5,103. 망원역 2번출구 앞,126.91083527,37.55495071,14,0,15,1
2,ST-6,104. 합정역 1번출구 앞,126.91508484,37.55073929,13,0,0,0
3,ST-7,105. 합정역 5번출구 앞,126.91482544,37.55000687,5,0,1,0
4,ST-8,106. 합정역 7번출구 앞,126.91282654,37.54864502,12,0,2,1


In [ ]:
bike_df_map.dtypes

stationId             object
stationName           object
stationLongitude      object
stationLatitude       object
rackTotCnt            object
parkingBikeTotCnt     object
parkingQRBikeCnt      object
parkingELECBikeCnt    object
dtype: object

In [ ]:
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype(float)
bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype(float)

bike_df_map['rackTotCnt'] = bike_df_map['rackTotCnt'].astype(int)
bike_df_map['parkingBikeTotCnt'] = bike_df_map['parkingBikeTotCnt'].astype(int)
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype(int)
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype(int)

bike_df_map['total'] = bike_df_map['parkingBikeTotCnt'] + bike_df_map['parkingQRBikeCnt'] + bike_df_map['parkingELECBikeCnt']

In [ ]:
bike_df_map['total'].head()

0    12
1    16
2     0
3     1
4     3
Name: total, dtype: int64

In [ ]:
bike_df_map.dtypes

stationId              object
stationName            object
stationLongitude      float64
stationLatitude       float64
rackTotCnt              int64
parkingBikeTotCnt       int64
parkingQRBikeCnt        int64
parkingELECBikeCnt      int64
total                   int64
dtype: object

In [ ]:
bike_df_map.shape

(2733, 9)

In [ ]:
bike_map = folium.Map(location = [bike_df_map['stationLatitude'].mean(),
                                  bike_df_map['stationLongitude'].mean()],
                      zoom_start = 20)

In [ ]:
for index, data in bike_df_map.iterrows() :
    popup_str = '{} 일반 : {}대, QR : {}대, 새싹 : {}대, 총 : {}대'.format(
        data['stationName'], data['parkingBikeTotCnt'], data['parkingQRBikeCnt'],
        data['parkingELECBikeCnt'], data['total']
    )
    popup = folium.Popup(popup_str, max_width = 600)
    folium.Marker(location = [data['stationLatitude'],
                              data['stationLongitude']],
                  popup = popup).add_to(bike_map)

bike_map

Output hidden; open in https://colab.research.google.com to view.